## Installation (work on colab)

In [1]:
!apt-get install -y \
    libgl1-mesa-dev \
    libgl1-mesa-glx \
    libglew-dev \
    libosmesa6-dev \
    software-properties-common

!apt-get install -y patchelf
!pip install gym

!pip install free-mujoco-py
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1
!pip install imageio==2.4.1
!pip install -U colabgymrender
!pip install mujoco

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
software-properties-common is already the newest version (0.99.22.8).
The following additional packages will be installed:
  libegl-dev libgl-dev libgles-dev libgles1 libglu1-mesa libglu1-mesa-dev libglvnd-core-dev
  libglvnd-dev libglx-dev libopengl-dev libosmesa6
The following NEW packages will be installed:
  libegl-dev libgl-dev libgl1-mesa-dev libgl1-mesa-glx libgles-dev libgles1 libglew-dev
  libglu1-mesa libglu1-mesa-dev libglvnd-core-dev libglvnd-dev libglx-dev libopengl-dev libosmesa6
  libosmesa6-dev
0 upgraded, 15 newly installed, 0 to remove and 15 not upgraded.
Need to get 3,952 kB of archives.
After this operation, 18.7 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libglx-dev amd64 1.4.0-1 [14.1 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libgl-dev amd64 1.4.0-1 [101 kB]
Get:3 http://archive.ubuntu.com/ubuntu 

In [2]:
!pip install pfrl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.6/112.6 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pfrl: filename=pfrl-0.4.0-py3-none-any.whl size=155461 sha256=900ff10a2e6f28bb916e87440e5b0586b7f589fdb3a7bf8fede81a5ecccf10fa
  Stored in directory: /root/.cache/pip/wheels/22/4a/0f/a87cd1ae925086eb3a1b8759f620fcf48e47939fb082946c3b
Successfully built pfrl


In [3]:
!pip install gymnasium

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 5.5 MB/s eta 0:00:00


In [4]:
from typing import Optional, List
from copy import deepcopy

import functools
import numpy as np
from sklearn.utils import check_random_state

import gymnasium as gym
import gym.wrappers
from gym.spaces import Discrete, Box

import pfrl

In [5]:
from google.colab import drive
drive.mount('/content/drive')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Mounted at /content/drive


In [6]:
%cd /content/drive/MyDrive/class-learning-for-robot-decision-making/_final_project/codes

/content/drive/MyDrive/class-learning-for-robot-decision-making/_final_project/codes


# Original env

In [7]:
env_name = "Hopper-v3"
num_envs = 1
seed = 42

monitor = False
render = False

process_seeds = np.arange(num_envs) + seed * num_envs

In [8]:
def make_env(process_idx, test):
    env = gym.make(env_name)  #
    # env = gym.make(env_name, exclude_current_positions_from_observation=False)  #
    # Unwrap TimiLimit wrapper
    assert isinstance(env, gym.wrappers.TimeLimit)
    env = env.env
    # Use different random seeds for train and test envs
    process_seed = int(process_seeds[process_idx])
    env_seed = 2**32 - 1 - process_seed if test else process_seed
    env.seed(env_seed)
    # Cast observations to float32 because our model uses float32
    env = pfrl.wrappers.CastObservationToFloat32(env)
    # Normalize action space to [-1, 1]^n
    env = pfrl.wrappers.NormalizeActionSpace(env)
    if monitor:
        env = gym.wrappers.Monitor(env, "monitor")
    if render:
        env = pfrl.wrappers.Render(env)
    return env

In [9]:
env = make_env(0, test=False)

/usr/local/lib/python3.10/dist-packages/gym/envs/registration.py:593: UserWarning: WARN: The environment Hopper-v3 is out of date. You should consider upgrading to version `v4`.
  logger.warn(


Compiling /usr/local/lib/python3.10/dist-packages/mujoco_py/cymj.pyx because it changed.
[1/1] Cythonizing /usr/local/lib/python3.10/dist-packages/mujoco_py/cymj.pyx


INFO:root:running build_ext
INFO:root:building 'mujoco_py.cymj' extension
INFO:root:creating /usr/local/lib/python3.10/dist-packages/mujoco_py/generated/_pyxbld_2.0.2.13_310_linuxcpuextensionbuilder
INFO:root:creating /usr/local/lib/python3.10/dist-packages/mujoco_py/generated/_pyxbld_2.0.2.13_310_linuxcpuextensionbuilder/temp.linux-x86_64-cpython-310
INFO:root:creating /usr/local/lib/python3.10/dist-packages/mujoco_py/generated/_pyxbld_2.0.2.13_310_linuxcpuextensionbuilder/temp.linux-x86_64-cpython-310/usr
INFO:root:creating /usr/local/lib/python3.10/dist-packages/mujoco_py/generated/_pyxbld_2.0.2.13_310_linuxcpuextensionbuilder/temp.linux-x86_64-cpython-310/usr/local
INFO:root:creating /usr/local/lib/python3.10/dist-packages/mujoco_py/generated/_pyxbld_2.0.2.13_310_linuxcpuextensionbuilder/temp.linux-x86_64-cpython-310/usr/local/lib
INFO:root:creating /usr/local/lib/python3.10/dist-packages/mujoco_py/generated/_pyxbld_2.0.2.13_310_linuxcpuextensionbuilder/temp.linux-x86_64-cpython-31

In [10]:
env.observation_space

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Box(-inf, inf, (11,), float64)

In [11]:
env.action_space

Box(-1.0, 1.0, (3,), float32)

In [12]:
obs_size = env.observation_space.shape[0]
action_size = env.action_space.shape[0]
timestep_limit = env.spec.max_episode_steps

# Loading base envs and base policies

In [13]:
import torch
from torch import distributions, nn

from pfrl import experiments, replay_buffers, utils
from pfrl.nn.lmbda import Lambda

In [14]:
# default params
policy_output_scale = 1.0

policy_hidden_dim = 256
policy_lr = 3e-4

q_func_hidden_dim = 256
q_func_lr = 3e-4

temperature_optimizer_lr = 3e-4

buffer_size = 10 ** 6
replay_start_size = 10 ** 4
batch_size = 256
gamma = 0.99

# steps = 10 ** 6
steps = 5 * 10 ** 5

In [15]:
def squashed_diagonal_gaussian_head(x):
    assert x.shape[-1] == action_size * 2
    mean, log_scale = torch.chunk(x, 2, dim=1)
    log_scale = torch.clamp(log_scale, -20.0, 2.0)
    var = torch.exp(log_scale * 2)
    base_distribution = distributions.Independent(
        distributions.Normal(loc=mean, scale=torch.sqrt(var)), 1
    )
    # cache_size=1 is required for numerical stability
    return distributions.transformed_distribution.TransformedDistribution(
        base_distribution, [distributions.transforms.TanhTransform(cache_size=1)]
    )

In [16]:
def make_policy_with_optimizer():
    policy = nn.Sequential(
        nn.Linear(obs_size, policy_hidden_dim),
        nn.ReLU(),
        nn.Linear(policy_hidden_dim, policy_hidden_dim),
        nn.ReLU(),
        nn.Linear(policy_hidden_dim, action_size * 2),
        Lambda(squashed_diagonal_gaussian_head),
    )
    torch.nn.init.xavier_uniform_(policy[0].weight)
    torch.nn.init.xavier_uniform_(policy[2].weight)
    torch.nn.init.xavier_uniform_(policy[4].weight, gain=policy_output_scale)
    policy_optimizer = torch.optim.Adam(policy.parameters(), lr=policy_lr)
    return policy, policy_optimizer

In [17]:
def make_q_func_with_optimizer():
    q_func = nn.Sequential(
        pfrl.nn.ConcatObsAndAction(),
        nn.Linear(obs_size + action_size, q_func_hidden_dim),
        nn.ReLU(),
        nn.Linear(q_func_hidden_dim, q_func_hidden_dim),
        nn.ReLU(),
        nn.Linear(q_func_hidden_dim, 1),
    )
    torch.nn.init.xavier_uniform_(q_func[1].weight)
    torch.nn.init.xavier_uniform_(q_func[3].weight)
    torch.nn.init.xavier_uniform_(q_func[5].weight)
    q_func_optimizer = torch.optim.Adam(q_func.parameters(), lr=q_func_lr)
    return q_func, q_func_optimizer

In [18]:
policy, policy_optimizer = make_policy_with_optimizer()
q_func1, q_func1_optimizer = make_q_func_with_optimizer()
q_func2, q_func2_optimizer = make_q_func_with_optimizer()

In [19]:
rbuffer = replay_buffers.ReplayBuffer(buffer_size)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [20]:
agent = pfrl.agents.SoftActorCritic(
    policy,
    q_func1,
    q_func2,
    policy_optimizer,
    q_func1_optimizer,
    q_func2_optimizer,
    rbuffer,
    gamma=gamma,
    replay_start_size=replay_start_size,
    gpu=0 if torch.cuda.is_available() else -1,
    minibatch_size=batch_size,
    entropy_target=-action_size,
    temperature_optimizer_lr=temperature_optimizer_lr,
    act_deterministically=True,
)

In [21]:
base_policies = []
# for  key in ["standard", "cubblestone", "sandpaper"]:
for  key in ["standard", "cubblestone", "sandpaper"]:
    agent_ = deepcopy(agent)
    agent_.load(f"base_policies/{key}")
    base_policies.append(agent_)

In [22]:
def make_env(process_idx, test, xml):
    env = gym.make(env_name, exclude_current_positions_from_observation=False, xml_file=xml)  #
    # env = gym.make(env_name, exclude_current_positions_from_observation=False)  #
    # Unwrap TimiLimit wrapper
    assert isinstance(env, gym.wrappers.TimeLimit)
    env = env.env
    # Use different random seeds for train and test envs
    process_seed = int(process_seeds[process_idx])
    env_seed = 2**32 - 1 - process_seed if test else process_seed
    env.seed(env_seed)
    # Cast observations to float32 because our model uses float32
    env = pfrl.wrappers.CastObservationToFloat32(env)
    # Normalize action space to [-1, 1]^n
    env = pfrl.wrappers.NormalizeActionSpace(env)
    if monitor:
        env = gym.wrappers.Monitor(env, "monitor")
    if render:
        env = pfrl.wrappers.Render(env)
    return env

In [23]:
base_envs = []
xml_files = ["hopper.xml", "envs/cobblestone_hopper_loose_v1.xml", "envs/sandpaper_hopper.xml"]
for i, xml in enumerate(xml_files):
    env_ = make_env(0, 0, xml)
    base_envs.append(env_)

/usr/local/lib/python3.10/dist-packages/gym/envs/registration.py:593: UserWarning: WARN: The environment Hopper-v3 is out of date. You should consider upgrading to version `v4`.
  logger.warn(
/usr/local/lib/python3.10/dist-packages/gym/envs/mujoco/mujoco_env.py:237: UserWarning: WARN: This version of the mujoco environments depends on the mujoco-py bindings, which are no longer maintained and may stop working. Please upgrade to the v4 versions of the environments (which depend on the mujoco python bindings instead), unless you are trying to precisely replicate previous works).
  logger.warn(
/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initi

In [24]:
base_policies

In [25]:
base_envs

[<NormalizeActionSpace<CastObservationToFloat32<OrderEnforcing<StepAPICompatibility<PassiveEnvChecker<HopperEnv<Hopper-v3>>>>>>>,
 <NormalizeActionSpace<CastObservationToFloat32<OrderEnforcing<StepAPICompatibility<PassiveEnvChecker<HopperEnv<Hopper-v3>>>>>>>,
 <NormalizeActionSpace<CastObservationToFloat32<OrderEnforcing<StepAPICompatibility<PassiveEnvChecker<HopperEnv<Hopper-v3>>>>>>>]

# Meta hopper env with terrain

In [26]:
def make_env(process_idx, test):
    env = gym.make(env_name)  #
    # env = gym.make(env_name, exclude_current_positions_from_observation=False)  #
    # Unwrap TimiLimit wrapper
    assert isinstance(env, gym.wrappers.TimeLimit)
    env = env.env
    # Use different random seeds for train and test envs
    process_seed = int(process_seeds[process_idx])
    env_seed = 2**32 - 1 - process_seed if test else process_seed
    env.seed(env_seed)
    # Cast observations to float32 because our model uses float32
    env = pfrl.wrappers.CastObservationToFloat32(env)
    # Normalize action space to [-1, 1]^n
    # env = pfrl.wrappers.NormalizeActionSpace(env)
    if monitor:
        env = gym.wrappers.Monitor(env, "monitor")
    if render:
        env = pfrl.wrappers.Render(env)
    return env

In [27]:
class HopperEnvWithTerrain(gym.Env):

    def __init__(
        self,
        base_envs: List[gym.Env] = base_envs,
        base_policies: Optional[List[pfrl.agent.Agent]] = None,
        average_distance_of_each_terrain: float = 1.0,
        dim_terrain_feature: int = 5,
        random_state: Optional[int] = None,
    ):
        if random_state is not None:
            self.random_ = check_random_state(random_state)
        else:
            self.random_ = check_random_state(12345)

        self.base_envs = base_envs
        self.base_policies = base_policies
        self.use_base_policy = (base_policies is not None)

        self.n_envs = len(base_envs)

        if base_policies is not None:
            self.n_policies = len(base_policies)
        else:
            self.n_policies = None

        self.base_terrain_feats = self.random_.uniform(
            size=(self.n_envs, dim_terrain_feature),
        )

        self.observation_space = Box(
            low=np.concatenate(
                (np.zeros((dim_terrain_feature)),
                 self.base_envs[0].observation_space.low)),
            high=np.concatenate(
                (np.ones((dim_terrain_feature)),
                 self.base_envs[0].observation_space.high)),
        )
        self.base_envs[0].observation_space
        self.dim_observation = self.observation_space.shape[0] - 1

        if self.use_base_policy:
            self.action_space = Discrete(3)
        else:
            self.action_space = self.base_envs[0].action_space

        self.average_distance_of_each_terrain = average_distance_of_each_terrain

    def _get_env_id_given_position(self):
        env_id = np.argmax(self.env_change_position > self.position)
        env_id = self.env_ids_position[env_id]
        return env_id

    def _update_state_of_all_envs(
        self,
        target_env_id: int,
    ):
        current_env = self.base_envs[target_env_id]
        qpos = current_env.sim.data.qpos
        qvel = current_env.sim.data.qvel
        for i in range(self.n_envs):
            self.base_envs[i].set_state(qpos, qvel)

    def reset(self):
        self.env_change_position = self.random_.normal(
            loc=self.average_distance_of_each_terrain,
            scale=self.average_distance_of_each_terrain * 0.1,
            size=(10000, )
        ).cumsum()
        self.env_ids_position = self.random_.choice(
            self.n_envs,
            size=10000,
            replace=True,
        )

        for i in range(len(base_envs)):
            self.base_envs[i].reset()

        env_id = self.env_ids_position[0]
        initial_state = self.base_envs[env_id].reset()

        self._update_state_of_all_envs(env_id)
        self.position = initial_state[0]

        self.state = initial_state[1:]

        return initial_state[1:]

    def step(self, action: np.array):
        env_id = self._get_env_id_given_position()

        if self.use_base_policy:
            if action == -1:
                action = self.base_policies[env_id].act(self.state)
            else:
                action = self.base_policies[action].act(self.state)

        state, reward, done, info = self.base_envs[env_id].step(action)
        self._update_state_of_all_envs(env_id)

        self.position = state[0]
        self.state = state[1:]

        return state[1:], reward, done, info

    def render(self):
        env_id = self._get_env_id_given_position()
        return self.base_envs[env_id].render()


In [28]:
gym.register(
    id="Hopper-with-terrain-v3",
    entry_point=HopperEnvWithTerrain,
    max_episode_steps=timestep_limit,
)

## Initialize environment

In [29]:
env_name = "Hopper-with-terrain-v3"

In [30]:
env = make_env(0, test=False)

In [31]:
def make_batch_env(test):
    return pfrl.envs.MultiprocessVectorEnv(
        [
            functools.partial(make_env, idx, test)
            for idx, env in enumerate(range(num_envs))
        ]
    )

## online evaluation

In [32]:
from collections import defaultdict
logs_online = defaultdict(dict)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [33]:
eval_stats = experiments.eval_performance(
    env=make_batch_env(test=True),
    agent=base_policies[0],
    n_steps=None,
    n_episodes=30,
    max_episode_len=timestep_limit,
)
print(
    "n_runs: {} mean: {} median: {} stdev {}".format(
        30,
        eval_stats["mean"],
        eval_stats["median"],
        eval_stats["stdev"],
    )
)
logs_online["mixed"]["standard"] = {
    "mean": eval_stats["mean"],
    "std": eval_stats["stdev"],
}

/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/core.py:256: DeprecationWarning: WARN: Function `env.seed(seed)` is marked as deprecated and will be removed in the future. Please use `env.reset(seed=seed)` instead.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:174: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed a `seed` in

n_runs: 30 mean: 405.2256423905689 median: 437.7032262341538 stdev 187.41018099161647


In [34]:
eval_stats = experiments.eval_performance(
    env=make_batch_env(test=True),
    agent=base_policies[1],
    n_steps=None,
    n_episodes=30,
    max_episode_len=timestep_limit,
)
print(
    "n_runs: {} mean: {} median: {} stdev {}".format(
        30,
        eval_stats["mean"],
        eval_stats["median"],
        eval_stats["stdev"],
    )
)
logs_online["mixed"]["cubblestone"] = {
    "mean": eval_stats["mean"],
    "std": eval_stats["stdev"],
}

/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/core.py:256: DeprecationWarning: WARN: Function `env.seed(seed)` is marked as deprecated and will be removed in the future. Please use `env.reset(seed=seed)` instead.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:174: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed a `seed` in

n_runs: 30 mean: 785.6243761447503 median: 675.6791236191839 stdev 444.4318419171776


In [35]:
eval_stats = experiments.eval_performance(
    env=make_batch_env(test=True),
    agent=base_policies[2],
    n_steps=None,
    n_episodes=30,
    max_episode_len=timestep_limit,
)
print(
    "n_runs: {} mean: {} median: {} stdev {}".format(
        10,
        eval_stats["mean"],
        eval_stats["median"],
        eval_stats["stdev"],
    )
)
logs_online["mixed"]["sandpaper"] = {
    "mean": eval_stats["mean"],
    "std": eval_stats["stdev"],
}

/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/core.py:256: DeprecationWarning: WARN: Function `env.seed(seed)` is marked as deprecated and will be removed in the future. Please use `env.reset(seed=seed)` instead.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:174: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed a `seed` in

n_runs: 10 mean: 449.9894968832533 median: 493.6201276227013 stdev 231.7963421608445


In [36]:
logs_online

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


defaultdict(dict,
            {'mixed': {'standard': {'mean': 405.2256423905689,
               'std': 187.41018099161647},
              'cubblestone': {'mean': 785.6243761447503,
               'std': 444.4318419171776},
              'sandpaper': {'mean': 449.9894968832533,
               'std': 231.7963421608445}}})

## online evaluation

In [42]:
terrains = ["standard", "cubblestone", "sandpaper"]
env_name = "Hopper-v3"

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [55]:
i = 2
xml = xml_files[i]
terrain = terrains[i]

In [56]:
def make_env(process_idx, test, xml):
    env = gym.make(env_name, xml_file=xml)  #
    # env = gym.make(env_name, exclude_current_positions_from_observation=False)  #
    # Unwrap TimiLimit wrapper
    assert isinstance(env, gym.wrappers.TimeLimit)
    env = env.env
    # Use different random seeds for train and test envs
    process_seed = int(process_seeds[process_idx])
    env_seed = 2**32 - 1 - process_seed if test else process_seed
    env.seed(env_seed)
    # Cast observations to float32 because our model uses float32
    env = pfrl.wrappers.CastObservationToFloat32(env)
    # Normalize action space to [-1, 1]^n
    env = pfrl.wrappers.NormalizeActionSpace(env)
    if monitor:
        env = gym.wrappers.Monitor(env, "monitor")
    if render:
        env = pfrl.wrappers.Render(env)
    return env

In [57]:
def make_batch_env(test):
    return pfrl.envs.MultiprocessVectorEnv(
        [
            functools.partial(make_env, idx, test, xml)
            for idx, env in enumerate(range(num_envs))
        ]
    )

In [58]:
eval_stats = experiments.eval_performance(
    env=make_batch_env(test=True),
    agent=base_policies[0],
    n_steps=None,
    n_episodes=30,
    max_episode_len=timestep_limit,
)
print(
    "n_runs: {} mean: {} median: {} stdev {}".format(
        10,
        eval_stats["mean"],
        eval_stats["median"],
        eval_stats["stdev"],
    )
)
logs_online[terrain]["standard"] = {
    "mean": eval_stats["mean"],
    "std": eval_stats["stdev"],
}

/usr/local/lib/python3.10/dist-packages/gym/envs/registration.py:593: UserWarning: WARN: The environment Hopper-v3 is out of date. You should consider upgrading to version `v4`.
  logger.warn(
/usr/local/lib/python3.10/dist-packages/gym/envs/mujoco/mujoco_env.py:237: UserWarning: WARN: This version of the mujoco environments depends on the mujoco-py bindings, which are no longer maintained and may stop working. Please upgrade to the v4 versions of the environments (which depend on the mujoco python bindings instead), unless you are trying to precisely replicate previous works).
  logger.warn(
/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initi

n_runs: 10 mean: 469.20446145478263 median: 468.26754188636386 stdev 3.4280574568066084


In [59]:
eval_stats = experiments.eval_performance(
    env=make_batch_env(test=True),
    agent=base_policies[1],
    n_steps=None,
    n_episodes=10,
    max_episode_len=timestep_limit,
)
print(
    "n_runs: {} mean: {} median: {} stdev {}".format(
        10,
        eval_stats["mean"],
        eval_stats["median"],
        eval_stats["stdev"],
    )
)
logs_online[terrain]["cubblestone"] = {
    "mean": eval_stats["mean"],
    "std": eval_stats["stdev"],
}

/usr/local/lib/python3.10/dist-packages/gym/envs/registration.py:593: UserWarning: WARN: The environment Hopper-v3 is out of date. You should consider upgrading to version `v4`.
  logger.warn(
/usr/local/lib/python3.10/dist-packages/gym/envs/mujoco/mujoco_env.py:237: UserWarning: WARN: This version of the mujoco environments depends on the mujoco-py bindings, which are no longer maintained and may stop working. Please upgrade to the v4 versions of the environments (which depend on the mujoco python bindings instead), unless you are trying to precisely replicate previous works).
  logger.warn(
/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initi

n_runs: 10 mean: 366.3825324353079 median: 371.50474072368684 stdev 9.693255399034443


In [60]:
eval_stats = experiments.eval_performance(
    env=make_batch_env(test=True),
    agent=base_policies[2],
    n_steps=None,
    n_episodes=10,
    max_episode_len=timestep_limit,
)
print(
    "n_runs: {} mean: {} median: {} stdev {}".format(
        10,
        eval_stats["mean"],
        eval_stats["median"],
        eval_stats["stdev"],
    )
)
logs_online[terrain]["sandpaper"] = {
    "mean": eval_stats["mean"],
    "std": eval_stats["stdev"],
}

/usr/local/lib/python3.10/dist-packages/gym/envs/registration.py:593: UserWarning: WARN: The environment Hopper-v3 is out of date. You should consider upgrading to version `v4`.
  logger.warn(
/usr/local/lib/python3.10/dist-packages/gym/envs/mujoco/mujoco_env.py:237: UserWarning: WARN: This version of the mujoco environments depends on the mujoco-py bindings, which are no longer maintained and may stop working. Please upgrade to the v4 versions of the environments (which depend on the mujoco python bindings instead), unless you are trying to precisely replicate previous works).
  logger.warn(
/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initi

n_runs: 10 mean: 655.4640629554333 median: 654.4135028605807 stdev 4.604439186647083


In [62]:
logs_online

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


defaultdict(dict,
            {'mixed': {'standard': {'mean': 405.2256423905689,
               'std': 187.41018099161647},
              'cubblestone': {'mean': 785.6243761447503,
               'std': 444.4318419171776},
              'sandpaper': {'mean': 449.9894968832533,
               'std': 231.7963421608445}},
             'standard': {'standard': {'mean': 3129.3473165856917,
               'std': 299.134366508957},
              'cubblestone': {'mean': 2162.077600110411,
               'std': 14.098829458746469},
              'sandpaper': {'mean': 572.164331486659,
               'std': 157.105049220142}},
             'cubblestone': {'standard': {'mean': 258.32780301031397,
               'std': 66.22912280348753},
              'cubblestone': {'mean': 1772.5815711670314,
               'std': 630.7208583867306},
              'sandpaper': {'mean': 178.57455477663723,
               'std': 2.5953701977561954}},
             'sandpaper': {'standard': {'mean': 469.20446145478

In [64]:
import pickle
with open("base_policies.pkl", "wb") as f:
    pickle.dump(logs_online, f)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [72]:
import pandas as pd
df = pd.DataFrame()
df["base_policy"] = ["standard", "cobblestone", "sandpaper"]
df["mixed"] = [logs_online["mixed"][key]["mean"] for key in ["standard", "cubblestone", "sandpaper"]]
df["standard"] = [logs_online["standard"][key]["mean"] for key in ["standard", "cubblestone", "sandpaper"]]
df["cobblestone"] = [logs_online["cubblestone"][key]["mean"] for key in ["standard", "cubblestone", "sandpaper"]]
df["sandpaper"] = [logs_online["sandpaper"][key]["mean"] for key in ["standard", "cubblestone", "sandpaper"]]

In [73]:
df

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,base_policy,mixed,standard,cobblestone,sandpaper
0,standard,405.225642,3129.347317,258.327803,469.204461
1,cobblestone,785.624376,2162.077600,1772.581571,366.382532
2,sandpaper,449.989497,572.164331,178.574555,655.464063


## render (TODO)

In [ ]:
import matplotlib.pyplot as plt
from IPython import display as ipythondisplay

In [ ]:
def make_env_for_render():
    env = gym.make(env_name, render_mode='rgb_array')
    env = gym.wrappers.RecordEpisodeStatistics(env)
    env.seed(seed)
    env.action_space.seed(seed)
    env.observation_space.seed(seed)
    return env

In [ ]:
env_ = make_env_for_render()

In [ ]:
plt.axis('off')
done = False
obs = env_.reset()

i = 0
while not done:
    i += 1
    if i % 20 == 0:
        ipythondisplay.clear_output(wait=True)
        # print("At timestep = ", i)
        screen = env_.render()
        plt.imshow(screen[0])
        ipythondisplay.display(plt.gcf())

    action = agent.act(obs)
    obs, reward, done, info = env_.step(action)

    if done:
        break

In [65]:
import seaborn as sns

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [66]:
glue = sns.load_dataset("glue")

In [67]:
pivot_df = glue.pivot(index="Model", columns="Task", values="Score")

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [68]:
glue

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Model,Year,Encoder,Task,Score
0,ERNIE,2019,Transformer,CoLA,75.5
1,T5,2019,Transformer,CoLA,71.6
2,RoBERTa,2019,Transformer,CoLA,67.8
3,BERT,2018,Transformer,CoLA,60.5
4,BiLSTM+ELMo,2018,LSTM,CoLA,32.1
...,...,...,...,...,...
59,BERT,2018,Transformer,RTE,70.1
60,BiLSTM+ELMo,2018,LSTM,RTE,57.4
61,BiLSTM+CoVe,2017,LSTM,RTE,52.7
62,BiLSTM+Attn,2017,LSTM,RTE,58.4


In [69]:
pivot_df

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Task,CoLA,MNLI,MRPC,QNLI,QQP,RTE,SST-2,STS-B
Model,,,,,,,,
BERT,60.5,86.7,89.3,92.7,72.1,70.1,94.9,87.6
BiLSTM,11.6,65.6,81.8,74.6,62.5,57.4,82.8,70.3
BiLSTM+Attn,18.6,67.6,83.9,74.3,60.1,58.4,83.0,72.8
BiLSTM+CoVe,18.5,65.4,78.7,70.8,60.6,52.7,81.9,64.4
BiLSTM+ELMo,32.1,67.2,84.7,75.5,61.1,57.4,89.3,70.3
ERNIE,75.5,92.3,93.9,97.3,75.2,92.6,97.8,93.0
RoBERTa,67.8,90.8,92.3,95.4,74.3,88.2,96.7,92.2
T5,71.6,92.2,92.8,96.9,75.1,92.8,97.5,93.1
